In [4]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 

from pathlib import Path
from tqdm import tqdm

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.inspection import permutation_importance

from functools import partial

#  model
from xgboost import XGBRegressor

#  data transformations
from src.data.data_utils import augment_data, fill_random_2d, transform_x

# scores and metrics
from sklearn.metrics import r2_score, make_scorer
from src.metrics.metrics import (exponential_mae, exponential_mape, exponential_mse, exponential_r2, 
exponential_mae_per_class, exponential_mape_per_class, exponential_mse_per_class, exponential_r2_per_class, r2_per_class)

%matplotlib qt
tqdm = partial(tqdm, position=0, leave=True)

In [5]:

# Paths
datasets_dir = Path('../../data/datasets/dataset_hplc_multi/')
reports_dir_test = Path('../../reports/cross_val')
reports_dir_train = Path('../../reports/cross_val_train')
dir_model = Path('../../model')
reports_dir_test.mkdir(parents=True, exist_ok=True)
reports_dir_train.mkdir(parents=True, exist_ok=True)
dir_model.mkdir(parents=True, exist_ok=True)

# params
pigments_threshold = [0.00248, 0.05878, 0.003  , 0.00518, 0.003  , 0.01302, 0.0036 , 0.00968, 0.001  , 0.0018 , 0.00844, 0.00242, 0.001]
log_pigments_threshold = list(np.log(pigments_threshold))


pigments = ['chlide_a[mg*m^3]', 'chla[mg*m^3]', 'chlb[mg*m^3]', 'chlc1+c2[mg*m^3]',
       'fucox[mg*m^3]', "19'hxfcx[mg*m^3]", "19'btfcx[mg*m^3]",
       'diadino[mg*m^3]', 'allox[mg*m^3]', 'diatox[mg*m^3]', 'zeaxan[mg*m^3]',
       'beta_car[mg*m^3]', 'peridinin[mg*m^3]']


param_dist = {
    'n_estimators': [50, 100, 200, 500, 1000, 1500, 2000],
    'max_depth': [2, 3, 5, 7, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.5, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1, 10],  # L1 regularization
    'reg_lambda': [1, 0.01, 0.1, 10, 100],  # L2 regularization
    'min_child_weight': [1, 2, 3, 5, 10]
}

param_dist= {
    'n_estimators': np.arange(50, 300, 25),       # Number of boosting rounds
    'max_depth': [3, 4, 5, 6, 7, 8, 10],          # Maximum depth of a tree
    'eta': [0.01, 0.05, 0.1, 0.2, 0.3],           # Learning rate (aka learning_rate)
    'colsample_bytree': [0.5, 0.6, 0.75, 0.9, 1.0],  # Feature subsample ratio per tree
    'min_child_weight': [1, 2, 3, 4, 5, 6, 10],    # Minimum sum of instance weight (hessian)
    'subsample': [0.5, 0.6, 0.75, 0.9, 1.0],       # Row subsample ratio
    'gamma': [0, 0.01, 0.1, 0.3, 0.5],             # Minimum loss reduction to split
    'lambda': [0.5, 1.0, 1.5, 2.0],                # L2 regularization
    'alpha': [0, 0.01, 0.1, 0.5, 1.0],             # L1 regularization
    'booster': ['gbtree', 'dart'],                # Type of booster to use
    'tree_method': ['auto'],                      # Can consider 'hist' for large datasets
}


metrics = {"mae": exponential_mae, "mse": exponential_mse, "r2": exponential_r2, "mape": exponential_mape,
           "mae_per_class": exponential_mae_per_class, "mse_per_class": exponential_mse_per_class, 
           "r2_per_class": exponential_r2_per_class, "mape_per_class": exponential_mape_per_class,
           "r2_log": r2_score, "r2_log_per_class": r2_per_class}

In [6]:

def ncv_xgboost(x, y, param_d, score, score_aux=None, outer_splits=5, inner_splits=3, seed=1, n_it=50):
    kfold_outer = KFold(n_splits=outer_splits, shuffle=True, random_state=seed)
    train_result = {}
    test_result = {}
    i = 0    
    for train_idx, test_idx in tqdm(kfold_outer.split(x), total=outer_splits):
        X_train, X_test = x.iloc[train_idx].copy(), x.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        X_train, y_train = augment_data(X_train, y_train, replicate=5)
        X_train.loc[:, ['lat', 'lon']] = fill_random_2d(X_train.loc[:, ['lat', 'lon']].values, 0.1)
        #X_train.loc[:, :] = fill_random_2d(X_train.loc[:, :].values, 0.1)

        kfold_inner = KFold(n_splits=inner_splits, shuffle=True, random_state=seed)
        
        # RandomizedSearchCV for hyperparameter tuning
        model = XGBRegressor(random_state=seed, verbosity=0)
        randomized_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_d,
            n_iter=n_it, 
            scoring=score,
            cv=kfold_inner,
            # cv=inner_splits,
            random_state=seed,
            n_jobs=-1
        )
    
        # Fit RandomizedSearchCV
        randomized_search.fit(X_train, y_train)
    
        # Get the best hyperparameters
        best_params = randomized_search.best_params_
    
        # Train a new model on the entire outer training set using the best hyperparameters
        best_model = XGBRegressor(random_state=seed, verbosity=0, **best_params)
        best_model.fit(X_train, y_train)
    
        # Evaluate the model on the outer test set
        aux_res_train = {}
        py = best_model.predict(X_train)
        if score_aux is not None:
            aux_res_train = {key: s(y_train, py) for key, s in score_aux.items()}
        train_result[i] = {**best_params, "score": score(best_model, X_train, y_train), **aux_res_train, "mean decrease impurity": best_model.feature_importances_}
    
        # Evaluate the model on the outer test set
        aux_res_test = {}
        py = best_model.predict(X_test)


        if score_aux is not None:
            aux_res_test.update({key: s(y_test, py) for key, s in score_aux.items()})
        pi = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2, scoring=score)
        test_result[i] = { **best_params, "score": score(best_model, X_test, y_test), **aux_res_test, 
                    "permutation importance mean": pi.importances_mean,
                    "permutation importance std": pi.importances_std,}
        i = i+1
    return train_result, test_result

In [7]:
x = pd.read_csv(datasets_dir/'log_rrs_lat_lon_month_season_depth_loc.csv')
y = pd.read_csv(datasets_dir/'log_pigments.csv')

# only med and black sea
y = y[x['med and black sea'].astype(bool)]
x = x[x['med and black sea'].astype(bool)]
x = x.drop(columns=['med', 'black sea', 'med and black sea'])


n = 50

# Wrap the custom score function for use in RandomizedSearchCV
# custom_scorer = make_scorer(custom_score, greater_is_better=True)
custom_scorer = make_scorer(r2_score, greater_is_better=True)

In [8]:
x = x.drop(columns=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
# x = x.drop(columns=['lat', 'lon'])
x = x.drop(columns=[ 'depth'])
x = x.drop(columns=['summer', 'autumn', 'spring', 'winter'])

In [9]:
'''data = pd.read_csv('../Pierre/data/pigments_export.csv')
pft_names = ['Bacillariophyceae', 'Bolidophyceae', 'Chlorarachnida', 'Chrysophyceae',
       'Coscinodiscophyceae', 'Cryptophyta', 'Dictyochophyceae',
       'Dinoflagellata', 'Haptophyta', 'Mediophyceae', 'MOCH',
       'Other.Photosynthetic.Eukaryotes', 'Pelagophyceae', 'Radiolaria',
       'Non.Phototrophic.Eukaryotes', 'Synechococcus', 'Prochlorococcus',
       'Non.Phototrophic.Prokaryotes', 'Other.Photosynthetic.Prokaryotes',
       'Cyanobium']

pig_names = ['Chl.C2', 'Peridinine', 'BF.19', 'Fucoxanthin',
       'Prasinoxanthin', 'HF.19', 'Diadinoxanthin', 'Alloxanthin',
       'Zeaxanthin', 'Chl.b', 'Chl.a', 'BB.Carotene']

x_raw = data[pig_names].copy()
y_raw = data[pft_names].copy()'''

"data = pd.read_csv('../Pierre/data/pigments_export.csv')\npft_names = ['Bacillariophyceae', 'Bolidophyceae', 'Chlorarachnida', 'Chrysophyceae',\n       'Coscinodiscophyceae', 'Cryptophyta', 'Dictyochophyceae',\n       'Dinoflagellata', 'Haptophyta', 'Mediophyceae', 'MOCH',\n       'Other.Photosynthetic.Eukaryotes', 'Pelagophyceae', 'Radiolaria',\n       'Non.Phototrophic.Eukaryotes', 'Synechococcus', 'Prochlorococcus',\n       'Non.Phototrophic.Prokaryotes', 'Other.Photosynthetic.Prokaryotes',\n       'Cyanobium']\n\npig_names = ['Chl.C2', 'Peridinine', 'BF.19', 'Fucoxanthin',\n       'Prasinoxanthin', 'HF.19', 'Diadinoxanthin', 'Alloxanthin',\n       'Zeaxanthin', 'Chl.b', 'Chl.a', 'BB.Carotene']\n\nx_raw = data[pig_names].copy()\ny_raw = data[pft_names].copy()"

In [10]:
'''# substitute 0s by lowest value and apply logs
x_raw_aux = x_raw.copy()
y_raw_aux = y_raw.copy()

x_zer_ind = x_raw_aux == 0
y_zer_ind = y_raw_aux == 0

x_raw_aux[x_zer_ind] = 1
y_raw_aux[y_zer_ind] = 1

x_raw = pd.DataFrame(np.maximum(x_raw.values, x_raw_aux.quantile(0.01)), columns=x_raw.columns)
y_raw = pd.DataFrame(np.maximum(y_raw.values, y_raw_aux.quantile(0.01)), columns=y_raw.columns)
x = np.log(x_raw)
y = np.log(y_raw)
'''

'# substitute 0s by lowest value and apply logs\nx_raw_aux = x_raw.copy()\ny_raw_aux = y_raw.copy()\n\nx_zer_ind = x_raw_aux == 0\ny_zer_ind = y_raw_aux == 0\n\nx_raw_aux[x_zer_ind] = 1\ny_raw_aux[y_zer_ind] = 1\n\nx_raw = pd.DataFrame(np.maximum(x_raw.values, x_raw_aux.quantile(0.01)), columns=x_raw.columns)\ny_raw = pd.DataFrame(np.maximum(y_raw.values, y_raw_aux.quantile(0.01)), columns=y_raw.columns)\nx = np.log(x_raw)\ny = np.log(y_raw)\n'

In [11]:
ncv_train, ncv_test  = ncv_xgboost(x, y, param_dist, custom_scorer, score_aux=metrics, n_it=n)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [31:59<00:00, 383.95s/it]


In [12]:
attribs_train = list(ncv_train[0].keys())
met_names_train = list(metrics.keys()) +['mean decrease impurity']

attribs_test = list(ncv_test[0].keys())
met_names_test = list(metrics.keys()) +['permutation importance mean', 'permutation importance std']

In [13]:
mets_train = {attrib : [fold[attrib] for fold_num, fold in ncv_train.items()] for attrib in attribs_train}
mets_train = {key: np.mean(value, axis=0)  if key in met_names_train else value for key, value in mets_train.items()}

mets_test = {attrib : [fold[attrib] for fold_num, fold in ncv_test.items()] for attrib in attribs_test}
mets_test = {key: np.mean(value, axis=0)  if key in met_names_test else value for key, value in mets_test.items()}

### Legacy training

In [14]:
x_mini = x.drop(columns=["400", "620", "510", "665", "681", "708", "778", "865"])

In [15]:
ncv_train_mini, ncv_test_mini = ncv_xgboost(x_mini, y, param_dist, custom_scorer, score_aux=metrics, n_it=n)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [17:07<00:00, 205.47s/it]


In [16]:
mets_train_mini = {attrib : [fold[attrib] for fold_num, fold in ncv_train_mini.items()] for attrib in attribs_train}
mets_train_mini = {key: np.mean(value, axis=0)  if key in met_names_train else value for key, value in mets_train_mini.items()}


In [17]:

mets_test_mini = {attrib : [fold[attrib] for fold_num, fold in ncv_test_mini.items()] for attrib in attribs_test}
mets_test_mini = {key: np.mean(value, axis=0)  if key in met_names_test else value for key, value in mets_test_mini.items()}

### Test Metrics

In [18]:
pd.DataFrame({'R2':[mets_test['r2_log'], mets_test_mini['r2_log']], 
              'MAPE':[mets_test['mape'], mets_test_mini['mape']],
              'MAE':[mets_test['mae'], mets_test_mini['mae']],
              'MSE':[mets_test['mse'], mets_test_mini['mse']]
             }, index=['RF', 'RF legacy'])

,R2,MAPE,MAE,MSE
RF,0.804524,0.427763,0.055045,0.134512
RF legacy,0.795455,0.429978,0.061940,0.172594


#### Per Pigment

In [19]:
pd.DataFrame([mets_test['r2_log_per_class'], mets_test_mini['r2_log_per_class']], columns=pigments, index=['RF', 'RF legacy'])

,chlide_a[mg*m^3],chla[mg*m^3],chlb[mg*m^3],chlc1+c2[mg*m^3],fucox[mg*m^3],19'hxfcx[mg*m^3],19'btfcx[mg*m^3],diadino[mg*m^3],allox[mg*m^3],diatox[mg*m^3],zeaxan[mg*m^3],beta_car[mg*m^3],peridinin[mg*m^3]
RF,0.749153,0.909602,0.765400,0.926696,0.903075,0.813327,0.626879,0.877600,0.728929,0.799256,0.611170,0.862095,0.885623
RF legacy,0.738921,0.901722,0.745471,0.911058,0.907490,0.801910,0.564929,0.886632,0.730049,0.776984,0.602307,0.876101,0.897336


### Training Metrics

In [21]:
pd.DataFrame({'R2':[mets_train['r2_log'], mets_train_mini['r2_log']], 
              'MAPE':[mets_train['mape'], mets_train_mini['mape']],
              'MAE':[mets_train['mae'], mets_train_mini['mae']],
              'MSE':[mets_train['mse'], mets_train_mini['mse']]
             }, index=['RF', 'RF legacy'])

,R2,MAPE,MAE,MSE
RF,0.996248,0.052552,0.006274,0.001275
RF legacy,0.994506,0.063382,0.007994,0.002103


In [22]:
pd.DataFrame([mets_train['r2_log_per_class'], mets_train_mini['r2_log_per_class']], columns=pigments, index=['RF', 'RF legacy'])

,chlide_a[mg*m^3],chla[mg*m^3],chlb[mg*m^3],chlc1+c2[mg*m^3],fucox[mg*m^3],19'hxfcx[mg*m^3],19'btfcx[mg*m^3],diadino[mg*m^3],allox[mg*m^3],diatox[mg*m^3],zeaxan[mg*m^3],beta_car[mg*m^3],peridinin[mg*m^3]
RF,0.997079,0.997446,0.996029,0.997943,0.998153,0.996201,0.993048,0.997737,0.996084,0.996391,0.990104,0.997321,0.997694
RF legacy,0.996238,0.995828,0.994293,0.996724,0.997205,0.994092,0.989558,0.996610,0.994428,0.994563,0.986513,0.995778,0.996746


## Save nested CV results

In [28]:
with open(reports_dir_train / 'xgboost.pkl', 'wb') as f:
    pickle.dump(mets_train, f)

with open(reports_dir_test / 'xgboost.pkl', 'wb') as f:
    pickle.dump(mets_test, f)

with open(reports_dir_train / 'xgboost_legacy.pkl', 'wb') as f:
    pickle.dump(mets_train_mini, f)

with open(reports_dir_test / 'xgboost_legacy.pkl', 'wb') as f:
    pickle.dump(mets_test_mini, f)

## Load nested CV results


In [29]:
with open(reports_dir_train / 'xgboost_legacy.pkl', 'rb') as f:
    mets_train_mini = pickle.load(f)

with open(reports_dir_test / 'xgboost_legacy.pkl', 'rb') as f:
    mets_test_mini = pickle.load(f)

with open(reports_dir_train / 'xgboost.pkl', 'rb') as f:
    mets_train = pickle.load(f)

with open(reports_dir_test / 'xgboost.pkl', 'rb') as f:
    mets_test = pickle.load(f)

## Show Feature importance

In [25]:
feature_names = x.columns

forest_importances = pd.Series(mets_test["permutation importance mean"], index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=mets_test["permutation importance std"], ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean r2 decrease")
fig.tight_layout()
plt.show()



In [30]:
feature_names = x_mini.columns

forest_importances = pd.Series(mets_test_mini["permutation importance mean"], index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=mets_test_mini["permutation importance std"], ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean r2 decrease")
fig.tight_layout()
plt.show()

### Train final model

In [31]:
with open(reports_dir_test / 'xgboost_legacy.pkl', 'rb') as f:
    mets_test_mini = pickle.load(f)

with open(reports_dir_test / 'xgboost.pkl', 'rb') as f:
    mets_test = pickle.load(f)

In [32]:
#  Select best hyperparameters

print("Hyperparameters are:\n")
hp = list(param_dist.keys())
hp

Hyperparameters are:



['n_estimators',
 'max_depth',
 'eta',
 'colsample_bytree',
 'min_child_weight',
 'subsample',
 'gamma',
 'lambda',
 'alpha',
 'booster',
 'tree_method']

In [33]:
print("Best Hyperparameters per Fold in outer loop")
pd.DataFrame([mets_test[hp_name] for hp_name in hp], 
             columns=["Fold 1", "Fold 2", "Fold 3", "Fold 4", "Fold 5"], 
             index=hp)

Best Hyperparameters per Fold in outer loop


,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5
n_estimators,250,250,250,250,250
max_depth,6,6,8,6,6
eta,0.05,0.05,0.05,0.05,0.05
colsample_bytree,0.9,0.9,0.9,0.9,0.9
min_child_weight,1,1,1,1,1
subsample,0.5,0.5,0.6,0.5,0.5
gamma,0.01,0.01,0.01,0.01,0.01
lambda,1.0,1.0,1.5,1.0,1.0
alpha,0,0,0.1,0,0
booster,dart,dart,gbtree,dart,dart


In [34]:
print("Best Hyperparameters per Fold in outer loop (legacy model)")
pd.DataFrame([mets_test_mini[hp_name] for hp_name in hp], 
             columns=["Fold 1", "Fold 2", "Fold 3", "Fold 4", "Fold 5"], 
             index=hp)

Best Hyperparameters per Fold in outer loop (legacy model)


,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5
n_estimators,250,250,250,250,250
max_depth,6,6,8,8,8
eta,0.05,0.05,0.05,0.05,0.05
colsample_bytree,0.9,0.9,0.9,0.9,0.9
min_child_weight,1,1,1,1,1
subsample,0.5,0.5,0.6,0.6,0.6
gamma,0.01,0.01,0.01,0.01,0.01
lambda,1.0,1.0,1.5,1.5,1.5
alpha,0,0,0.1,0.1,0.1
booster,dart,dart,gbtree,gbtree,gbtree


In [39]:
best_hp ={hp_name: mets_test[hp_name][0] for hp_name in hp}
best_hp

{'n_estimators': 250,
 'max_depth': 6,
 'eta': 0.05,
 'colsample_bytree': 0.9,
 'min_child_weight': 1,
 'subsample': 0.5,
 'gamma': 0.01,
 'lambda': 1.0,
 'alpha': 0,
 'booster': 'dart',
 'tree_method': 'auto'}

In [46]:
x_train, y_train = augment_data(x, y, replicate=5)
x_train.loc[:, ['lat', 'lon']] = fill_random_2d(x_train.loc[:, ['lat', 'lon']].values, 0.1)

best_model = XGBRegressor(**best_hp)
best_model.fit(x_train, y_train)

XGBRegressor(alpha=0, base_score=None, booster='dart', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.05, eval_metric=None,
             feature_types=None, gamma=0.01, grow_policy=None,
             importance_type=None, interaction_constraints=None, lambda=1.0,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=250, ...)

In [47]:
x_train_mini = x_train.drop(columns=["400", "620", "510", "665", "681", "708", "778", "865"])

best_model_legacy = XGBRegressor(**best_hp)
best_model_legacy.fit(x_train_mini, y_train)

XGBRegressor(alpha=0, base_score=None, booster='dart', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.05, eval_metric=None,
             feature_types=None, gamma=0.01, grow_policy=None,
             importance_type=None, interaction_constraints=None, lambda=1.0,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=250, ...)

In [48]:
r2_score(y.values, best_model.predict(x))

0.9920166135220365

In [49]:
best_model.feature_names_in_

array(['400', '412', '442', '490', '510', '560', '620', '665', '673',
       '681', '708', '778', '865', 'lat', 'lon'], dtype='<U3')

### Save models

In [50]:
with open(dir_model / 'xgb.pkl', 'wb') as f:
    pickle.dump(best_model, f)
    
with open(dir_model / 'xgb_legacy.pkl', 'wb') as f:
    pickle.dump(best_model_legacy, f)